In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


Using TensorFlow backend.


In [2]:
from IPython import get_ipython
from IPython.display import display

In [3]:
# Clone dev branch for the latest model
!git clone -b dev --recursive https://github.com/shaoanlu/fewshot-face-translation-GAN.git

fatal: destination path 'fewshot-face-translation-GAN' already exists and is not an empty directory.


In [ ]:
!pip install tensorflow==1.15 keras==2.2.5 --quiet


In [4]:
# There are import errors under keras == 2.2.5
!pip install keras==2.2.5

In [5]:
%cd fewshot-face-translation-GAN

/content/fewshot-face-translation-GAN


In [6]:
# Download pre-trined weights
!gdown https://drive.google.com/uc?id=1ThyUhXkVY5WJIRbik_Jpx9vOqEQ4jEZw
!gdown https://drive.google.com/uc?id=1cVkPjTxqQXo0pyLQHw_Yhngc4GO6KB5Q

Downloading...
From: https://drive.google.com/uc?id=1ThyUhXkVY5WJIRbik_Jpx9vOqEQ4jEZw
To: /content/fewshot-face-translation-GAN/encoder.h5
100% 6.26M/6.26M [00:00<00:00, 215MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1cVkPjTxqQXo0pyLQHw_Yhngc4GO6KB5Q
From (redirected): https://drive.google.com/uc?id=1cVkPjTxqQXo0pyLQHw_Yhngc4GO6KB5Q&confirm=t&uuid=a99b4ed7-ad3f-4ae2-b3ce-ad6d5e843a25
To: /content/fewshot-face-translation-GAN/decoder.h5
100% 126M/126M [00:00<00:00, 263MB/s]


In [7]:
!mkdir weights
!mv decoder.h5 weights/decoder.h5
!mv encoder.h5 weights/encoder.h5

mkdir: cannot create directory ‘weights’: File exists


In [8]:
!ls

colab_demo.ipynb    fewshot-face-translation-GAN  preprocess.py  utils
configs		    images			  __pycache__	 weights
data		    models.py			  README.md
face_toolbox_keras  networks			  train.py


## Configuration

In [9]:
from utils.config_loader import load_yaml
config = load_yaml("configs/config_inference.yaml"); config

{'dir_weights': './weights',
 'identity_extractor': 'inceptionresnetv1',
 'input_size': 224,
 'latent_dim': 512,
 'separate_adain': True,
 'additional_emb': True,
 'use_nwg': False,
 'nc_in': 3}

## Load GAN model

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
# Navigate to the models.py file in the cloned repository and modify it.
# Replace the line importing Beta from tensorflow.contrib
!sed -i "s/from tensorflow.contrib.distributions import Beta/from tensorflow.compat.v1.distributions import Beta/" /content/fewshot-face-translation-GAN/fewshot-face-translation-GAN/models.py

In [12]:
!find /content/fewshot-face-translation-GAN -type f -name "*.py" -exec sed -i 's/tf\.get_default_graph()/tf.compat.v1.get_default_graph()/g' {} \;

print("Replaced tf.get_default_graph() with tf.compat.v1.get_default_graph()")


Replaced tf.get_default_graph() with tf.compat.v1.get_default_graph()


In [13]:
from models import FaceTranslationGANInferenceModel

ImportError: cannot import name 'Iterable' from 'collections' (/usr/lib/python3.11/collections/__init__.py)

In [ ]:
model = FaceTranslationGANInferenceModel(config=config)

In [ ]:
!grep -r "get_default_graph" /content/


In [ ]:
from face_toolbox_keras.models.verifier.face_verifier import FaceVerifier
fv = FaceVerifier(classes=512)

from face_toolbox_keras.models.parser import face_parser
fp = face_parser.FaceParser()

from face_toolbox_keras.models.detector import face_detector
fd = face_detector.FaceAlignmentDetector()

from face_toolbox_keras.models.detector.iris_detector import IrisDetector
idet = IrisDetector()
#idet.set_detector(fd)

## Upload test data

In [ ]:
import numpy as np
from utils import utils
from matplotlib import pyplot as plt

In [ ]:
from google.colab import files

In [ ]:
# Upload a source face image.
# There should be only one source face image.
fn_src = files.upload()

In [ ]:
# Upload target face images.
# Number of target face images is not restricted as long as they belong to the same identity.
fns_tar = files.upload()

In [ ]:
# Set input image path
fn_src = [k for k,v in fn_src.items()]
if len(fn_src) >= 1:
    fn_src = fn_src[0]

fns_tar = [k for k,v in fns_tar.items()]

print(fn_src)
print(fns_tar)

In [ ]:
# We can also manually assign filenmes using the following code
#fn_src = "test02.jpg"
#fns_tar = ["test01.jpg", "test03.jpg", "test04.jpg", "test05.jpg"]

## Translate faces

### Inferece

It requires additional time to load models for the first infernce.

In [ ]:
src, mask, aligned_im, (x0, y0, x1, y1), landmarks = utils.get_src_inputs(fn_src, fd, fp, idet, identity_extractor=config["identity_extractor"])
tar, emb_tar = utils.get_tar_inputs(fns_tar, fd, fv, identity_extractor=config["identity_extractor"])

In [ ]:
out = model.inference(src, mask, tar, emb_tar)

### Visualize results

Images are resized to having maximum side length of 768.

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(src)
plt.subplot(1,2,2)
plt.imshow(tar)

In [ ]:
result_face = np.squeeze(((out[0] + 1) * 255 / 2).astype(np.uint8))
plt.imshow(result_face)

In [ ]:
result_img = utils.post_process_result(fn_src, fd, result_face, aligned_im, src, x0, y0, x1, y1, landmarks)
plt.imshow(result_img)